In [1]:
from z3 import BitVec, Optimize

In [2]:
data = open("input/17").read().splitlines()

In [3]:
rega = int(data[0].split(": ")[1])
regb = int(data[1].split(": ")[1])
regc = int(data[2].split(": ")[1])

regs = {"A": rega, "B": regb, "C": regc}

program = list(map(int, data[4].split(": ")[1].split(",")))

In [4]:
def get_combo_operand(op):
    if op == 4:
        return regs["A"]
    elif op == 5:
        return regs["B"]
    elif op == 6:
        return regs["C"]
    return op

In [5]:
pc = 0
output = []
while True:
    if pc >= len(program):
        break
    instr, op = program[pc], program[pc + 1]

    if instr == 0:
        regs["A"] = int(regs["A"] / 2 ** get_combo_operand(op))
        pc += 2

    elif instr == 1:
        regs["B"] = regs["B"] ^ op
        pc += 2

    elif instr == 2:
        regs["B"] = get_combo_operand(op) % 8
        pc += 2

    elif instr == 3:
        if regs["A"] == 0:
            pc += 2
            continue
        pc = op

    elif instr == 4:
        regs["B"] = regs["B"] ^ regs["C"]
        pc += 2
        
    elif instr == 5:
        output.append(get_combo_operand(op) % 8)
        pc += 2
        
    elif instr == 6:
        regs["B"] = int(regs["A"] / 2 ** get_combo_operand(op))
        pc += 2
        
    elif instr == 7:
        regs["C"] = int(regs["A"] / 2 ** get_combo_operand(op))
        pc += 2

In [6]:
part1 = ",".join(list(map(str, output)))
print(f"Answer #1: {part1}")

Answer #1: 4,3,2,6,4,5,3,2,4


# Part 2

In [7]:
# Got help here :-)
# After analyzing the operations, found some simplifications on operands
# Created conditions for a z3 solver :) 

In [8]:
pc = 0
output = []
symbolic_operations = []
while True:
    if pc >= len(program):
        break
    instr, op = program[pc], program[pc + 1]

    if instr == 0:
        symbolic_operations.append("A = A >> 3")
    elif instr == 1:
        symbolic_operations.append(f"B = B ^ {op}")
    elif instr == 2:
        symbolic_operations.append("B = A % 8")
    elif instr == 3:
        break
    elif instr == 4:
        symbolic_operations.append("B = B ^ C")
    elif instr == 7:
        symbolic_operations.append("C = A >> B")

    pc += 2

In [9]:
S = BitVec("A", 64)  # answer should be at least 48 (3 * len(output))
A = S
B = 0
C = 0

solver = Optimize()

In [10]:
for value in program:
    for op in symbolic_operations:
        dest, operation = op.split(" = ")
        if dest == "A":
            A = eval(operation)
        if dest == "B":
            B = eval(operation)
        if dest == "C":
            C = eval(operation)
    solver.add(B % 8 == value)
solver.add(A == 0)

In [11]:
solver.minimize(S)
solver.check()
part2 = solver.model()[S]
print(f"Answer #2: {part2}")

Answer #2: 164540892147389
